In [12]:
#-*-coding:utf8-*-
import pandas as pd
import numpy as np
import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [8]:
from pyltp import Segmentor
from pyltp import Postagger
from pyltp import NamedEntityRecognizer

In [9]:
segmentor = Segmentor()  # 初始化实例
segmentor.load('/home/sherlock/Documents/ltp_data/cws.model')
#实例化词性工具
postagger = Postagger() # 初始化实例
postagger.load('/home/sherlock/Documents/ltp_data/pos.model')  # 加载模型
recognizer = NamedEntityRecognizer()
recognizer.load('/home/sherlock/Documents/ltp_data/ner.model')

In [10]:
def wdseg(inputstr,ret_type):


    words = segmentor.segment(inputstr)  # 分词
    if ret_type == 'str':
        seg_word = ' '.join(words)
    if ret_type == 'lst':
        seg_word = ' '.join(words)
        seg_word = seg_word.split()

    #segmentor.release()  # 释放模型
    return seg_word

In [114]:
des_path = '/media/sherlock/new30/law/jun28InfoExtractor/tb_doc_add3.csv'

In [115]:
add3_law_df = pd.read_csv(des_path)
add3_law_df.info()

In [116]:
for dd,np in enumerate(add3_law_df.native_place.tolist()):
    if not isinstance(np,float) and len(np) >0:
        print dd
        print np
        print len(np)
        

In [4]:
import numpy as np

In [125]:
#be cautions on 867

In [118]:
print add3_law_df.loc[876,'native_place']

In [119]:
print add3_law_df.loc[876,'party_info']

In [65]:
info_words = wdseg( (add3_law_df.loc[977,'party_info']).encode('utf8'),'lst')

In [79]:
def get_native_palce(info_seg_list):
    '''提取籍贯，常见的句式是，籍贯XX市，出生于XX市，生于XX市，XX人，XX市出生
    输入：分词list,
    step1生成postaglist,
    step2找出逗号句号的index，这样可以分出句子，
    step3找出籍贯， 出生，人等关键词的index
    step4把signal_word's index匹配进第一步的句子中
    step5然后找出句子中的NS（地点）
    step6输出 地点组成的string'''
    def postaglist(info_seg_list):
        postag = postagger.postag(info_seg_list)
        postag_list = '\t'.join(postag).split('\t')
        return postag_list

    def comma_prd_index(info_seg_list):
        com_prd_idx = [idx for idx,v in enumerate(info_seg_list) if v=='，'or v =='。' or v==',' or v=='.']
        return com_prd_idx

    def find_deft_loc(info_seg_list):
        dft_index = [idx for idx,dft in enumerate(info_seg_list) if u'被告'in dft]
        return dft_index[0]

    def find_pltf_loc(info_seg_list):
        pltf_index = [idx for idx,pltf in enumerate(info_seg_list) if u'原告' in pltf]
        return pltf_index

    def sgl_word_index(info_seg_list):
        #唐山县人民法院会被认为是出生地
        #如果party_info中有原告XX，出生于XX，
        #被告人窦某某，47岁，出生于吉林省蛟河市，户籍所在地蛟河市，现住蛟河市
        jg_singal_words3 = u"籍"
        jg_singal_words2 = u"人"
        jg_singal_words1 = u"生"

        #jg_sgl_word_list = jg_singal_words.split(',')
        sgl_word_idx = [ ]
        #for index, sgl in enumerate(jg_sgl_word_list):
        for wd_idx,ww in enumerate(info_seg_list):
            if jg_singal_words1 in ww:
                sgl_word_idx.append(wd_idx)
        #jg_singal_words2 = u'人'
        #jg_sgl_word_list2 = jg_singal_words2.split()
        #for wd_idx2,ww in enumerate(info_seg_list):
            if jg_singal_words2 in ww:
                if info_seg_list[wd_idx+1] ==',' or info_seg_list[wd_idx+1]=='.'or info_seg_list[wd_idx+1]=='，' or info_seg_list[wd_idx+1]=='。':
                    sgl_word_idx.append(wd_idx)
            if jg_singal_words3 in ww:
                    sgl_word_idx.append(wd_idx)
        print 'inti_sgl_word index' ,sgl_word_idx

        dft_start = find_deft_loc(info_seg_list)
        sgl_word_start = np.searchsorted(sgl_word_idx,dft_start,'left')
        #add pltf index list, and dft start to insert into the list,
        #if the plft list's length is larger than zero and the dft inserted index is
        # less than the plft length ,    then add the next pltf index to the sgl_word_idx
        pltf_idx = find_pltf_loc(info_seg_list)
        if len(pltf_idx)>0 :
            dft_into_pltf_index = np.searchsorted(pltf_idx,dft_start,'left')
            if dft_into_pltf_index < len(pltf_idx):

                sgl_word_end = pltf_idx[dft_into_pltf_index]
                sgl_word_idx = sgl_word_idx[sgl_word_start:sgl_word_end]
            else:
                sgl_word_idx = sgl_word_idx[sgl_word_start:]

        else:

            sgl_word_idx = sgl_word_idx[sgl_word_start:]
        for ii in  sgl_word_idx:print info_seg_list[ii]
        return sgl_word_idx

    def sgl_range(sgl_word_index,comma_prd_index):
        sgl_range=[]
        for sg in sgl_word_index:
            start = np.searchsorted(comma_prd_index,sg,'left')
            print start
            print len(comma_prd_index)
            if start >0:
                start_range= comma_prd_index[start-1]
                if start <len(comma_prd_index)-1:
                    end_range = comma_prd_index[start]
                else:
                    end_range = comma_prd_index[-1]
                print 'start_range is' , start_range
                print 'end_range is',end_range
    #if start == 0:
       # start_range = 0
        #end_range = com_prd_idx[0]
                sgl_range.append((start_range,end_range))
            print sgl_range
        return sgl_range

    def find_loc(sql_range,postag_list,info_seg_list):
        jg_str = ''
        for i in sgl_range:
            jg_str2 = ''
            if 'ns' in postag_list[i[0]:i[1]]:
                for ix,pp in enumerate(postag_list[i[0]:i[1]]):
                    if pp == 'ns':
                        jg_ns =  info_seg_list[i[0]:i[1]][ix]
                        print jg_ns
                        jg_str2 = jg_str2+jg_ns
                        print jg_str2
            jg_str = jg_str + jg_str2 +' '
        return jg_str

    postag_list = postaglist(info_seg_list)
    com_prd_index = comma_prd_index(info_seg_list)
    sgl_index = sgl_word_index(info_seg_list)
    sgl_range = sgl_range(sgl_index, com_prd_index)
    jg_str = find_loc(sgl_range, postag_list, info_seg_list)
    return jg_str
    # the native_born_place is still ne

In [123]:
def get_native_palce(info_seg_list):
    '''提取籍贯，常见的句式是，籍贯XX市，出生于XX市，生于XX市，XX人，XX市出生
    输入：分词list,
    step1生成postaglist,
    step2找出逗号句号的index，这样可以分出句子，
    step3找出籍贯， 出生，人等关键词的index
    step4把signal_word's index匹配进第一步的句子中
    step5然后找出句子中的NS（地点）
    step6输出 地点组成的string'''
    def postaglist(info_seg_list):
        postag = postagger.postag(info_seg_list)
        postag_list = '\t'.join(postag).split('\t')
        return postag_list

    def comma_prd_index(info_seg_list):
        com_prd_idx = [idx for idx,v in enumerate(info_seg_list) if v=='，'or v =='。' or v==',' or v=='.']
        return com_prd_idx

    def find_deft_loc(info_seg_list):
        dft_index = [idx for idx,dft in enumerate(info_seg_list) if u'被告'in dft]
        return dft_index

    def find_pltf_loc(info_seg_list):
        pltf_index = [idx for idx,pltf in enumerate(info_seg_list) if u'原告' in pltf]
        return pltf_index

    def sgl_word_index(dft_start,info_seg_list):
        #唐山县人民法院会被认为是出生地
        #如果party_info中有原告XX，出生于XX，
        #被告人窦某某，47岁，出生于吉林省蛟河市，户籍所在地蛟河市，现住蛟河市
        jg_singal_words3 = u"籍"
        jg_singal_words2 = u"人"
        jg_singal_words1 = u"生"
        jg_singal_blackwords =u"人民"

        #jg_sgl_word_list = jg_singal_words.split(',')
        sgl_word_idx = [ ]
        #for index, sgl in enumerate(jg_sgl_word_list):
        for wd_idx,ww in enumerate(info_seg_list):
            if jg_singal_words1 in ww:
                sgl_word_idx.append(wd_idx)
        #jg_singal_words2 = u'人'
        #jg_sgl_word_list2 = jg_singal_words2.split()
        #for wd_idx2,ww in enumerate(info_seg_list):
            elif jg_singal_words2 in ww and not jg_singal_blackwords in ww :
                if info_seg_list[wd_idx+1] ==',' or info_seg_list[wd_idx+1]=='.'or info_seg_list[wd_idx+1]=='，' or info_seg_list[wd_idx+1]=='。':
                    sgl_word_idx.append(wd_idx)
            elif jg_singal_words3 in ww:
                    sgl_word_idx.append(wd_idx)
        print 'inti_sgl_word index' ,sgl_word_idx

        ##dft_start = find_deft_loc(info_seg_list)
        sgl_word_start = np.searchsorted(sgl_word_idx,dft_start,'left')
        #add pltf index list, and dft start to insert into the list,
        #if the plft list's length is larger than zero and the dft inserted index is
        # less than the plft length ,    then add the next pltf index to the sgl_word_idx
        pltf_idx = find_pltf_loc(info_seg_list)
        if len(pltf_idx)>0 :
            dft_into_pltf_index = np.searchsorted(pltf_idx,dft_start,'left')
            if dft_into_pltf_index < len(pltf_idx):

                sgl_word_end = pltf_idx[dft_into_pltf_index]
                sgl_word_idx = sgl_word_idx[sgl_word_start:sgl_word_end]
            else:
                sgl_word_idx = sgl_word_idx[sgl_word_start:]

        else:

            sgl_word_idx = sgl_word_idx[sgl_word_start:]
        if len(sgl_word_idx)>0:
            sgl_word_idx = sgl_word_idx[0]
        #print info_seg_list[sgl_word_idx]
        return sgl_word_idx

    def sgl_range(sgl_word_index,comma_prd_index):
        sgl_range=[]
        for sg in sgl_word_index:
            start = np.searchsorted(comma_prd_index,sg,'left')
            print start
            print len(comma_prd_index)
            if start >0:
                start_range= comma_prd_index[start-1]
                if start <len(comma_prd_index)-1:
                    end_range = comma_prd_index[start]
                else:
                    end_range = comma_prd_index[-1]
                print 'start_range is' , start_range
                print 'end_range is',end_range
    #if start == 0:
       # start_range = 0
        #end_range = com_prd_idx[0]
                sgl_range.append((start_range,end_range))
            print sgl_range
        return sgl_range

    def find_loc(sql_range,postag_list,info_seg_list):
        jg_str = ''
        for i in sgl_range:
            jg_str2 = ''
            if 'ns' in postag_list[i[0]:i[1]]:
                for ix,pp in enumerate(postag_list[i[0]:i[1]]):
                    if pp == 'ns':
                        jg_ns =  info_seg_list[i[0]:i[1]][ix]
                        print jg_ns
                        jg_str2 = jg_str2+jg_ns
                        print jg_str2
            jg_str = jg_str + jg_str2 +' '
        return jg_str

    postag_list = postaglist(info_seg_list)
    com_prd_index = comma_prd_index(info_seg_list)
    dft_start = find_deft_loc(info_seg_list)
    sgl_idx = []
    for dft in dft_start:

        sgl_index = sgl_word_index(dft, info_seg_list)
        sgl_idx.append(sgl_index)
    sgl_range = sgl_range(sgl_idx, com_prd_index)
    jg_str = find_loc(sgl_range, postag_list, info_seg_list)
    return jg_str

In [124]:
print add3_law_df.loc[977,'party_info']
nt_place_17 = get_native_palce(info_words)
print'native_palce final is ', nt_place_17

AttributeError: 'str' object has no attribute 'searchsorted'